<a href="https://colab.research.google.com/github/Prathamesh-Chavan-98/NLP-Sentense-Autocomplete/blob/main/_NLP_Sentense_Autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets
import opendatasets as od

In [2]:
od.download("https://www.kaggle.com/datasets/kingburrito666/shakespeare-plays")

Dataset URL: https://www.kaggle.com/datasets/kingburrito666/shakespeare-plays


100%|██████████| 4.55M/4.55M [00:00<00:00, 61.2MB/s]

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout , Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pickle
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('/content/shakespeare-plays/Shakespeare_data.csv')

In [5]:
df = data.copy()

In [6]:
df.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [7]:
# getting text from the data
text = []
for i in data['PlayerLine']:
	text.append(i)

# lets see how the text is looking
text[:5]


['ACT I',
 'SCENE I. London. The palace.',
 'Enter KING HENRY, LORD JOHN OF LANCASTER, the EARL of WESTMORELAND, SIR WALTER BLUNT, and others',
 'So shaken as we are, so wan with care,',
 'Find we a time for frighted peace to pant,']

In [9]:
import re # import the 're' module for regular expression operations

# Text Cleaning
def clean_text(text):
    # removing special characters like @, #, $, etc
    pattern = re.compile('[^a-zA-z0-9\s]')
    text = re.sub(pattern,'',text)

    # removing digits
    pattern = re.compile('\d+')
    text = re.sub(pattern,'',text)

    # converting text to lower case
    text = text.lower()
    return text

texts = []
for t in text:
    new_text = clean_text(t)
    texts.append(new_text)

# cleaned text
texts[:5]

['act i',
 'scene i london the palace',
 'enter king henry lord john of lancaster the earl of westmoreland sir walter blunt and others',
 'so shaken as we are so wan with care',
 'find we a time for frighted peace to pant']

In [11]:
import re # import the 're' module for regular expression operations
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# using tensorflow tokenizer and
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# saving the tokenizer for the inference phase
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
# generating text sequences, i.e. encoding the text
text_sequences = tokenizer.texts_to_sequences(texts) # Remove np.array() here

# padding the sequences BEFORE converting to numpy array
Max_Sequence_Len = max([len(x) for x in text_sequences])
text_sequences = pad_sequences(text_sequences,
                            maxlen = Max_Sequence_Len, padding='pre')

# Now convert to numpy array after padding
text_sequences = np.array(text_sequences)

print('Text -->>',texts[0])
print('Embedding -->>',text_sequences[0])

print('Maximum Sequence Length -->>',Max_Sequence_Len)
print('Text Sequence -->>\n',text_sequences[0])
print('Text Sequence Shape -->>',text_sequences.shape)

Text -->> act i
Embedding -->> [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 455   4]
Maximum Sequence Length -->> 54
Text Sequence -->>
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 455   4]
Text Sequence Shape -->> (10000, 54)


In [12]:
# getting X and y from the data
X, y = text_sequences[:, :-1], text_sequences[:,-1]
print('First Input :',X[0])
print('First Target :',y[0])

word_index = tokenizer.word_index

# using one hot encoding on y
total_words = len(word_index) + 1
print('Total Number of Words:',total_words)

y = to_categorical(y, num_classes=total_words)

# printing X and y shapes
print('Input Shape :',X.shape)
print('Target Shape :',y.shape)


First Input : [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 455]
First Target : 4
Total Number of Words: 7865
Input Shape : (10000, 53)
Target Shape : (10000, 7865)


In [13]:
model = Sequential(name="LSTM_Model")

# adding embedding layer
model.add(Embedding(total_words,
					Max_Sequence_Len-1,
					input_length=Max_Sequence_Len-1))

# adding a LSTM layer
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.5))

#adding the final output activation with activation function of softmax
model.add(Dense(total_words, activation='softmax'))

# printing model summary
print(model.summary())


Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 53, 53)            416845    
                                                                 
 lstm (LSTM)                 (None, 512)               1159168   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 7865)              4034745   
                                                                 
Total params: 5610758 (21.40 MB)
Trainable params: 5610758 (21.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
# Compiling the model
model.compile(
	loss="categorical_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

# Training the LSTM model
history = model.fit(X, y,
					epochs=50,
					verbose=1)


Epoch 1/50
313/313 [==============================] - 17s 41ms/step - loss: 7.9185 - accuracy: 0.0111
Epoch 2/50
313/313 [==============================] - 5s 17ms/step - loss: 7.3621 - accuracy: 0.0176
Epoch 3/50
313/313 [==============================] - 5s 18ms/step - loss: 7.1087 - accuracy: 0.0239
Epoch 4/50
313/313 [==============================] - 5s 15ms/step - loss: 6.8595 - accuracy: 0.0288
Epoch 5/50
313/313 [==============================] - 5s 16ms/step - loss: 6.5970 - accuracy: 0.0330
Epoch 6/50
313/313 [==============================] - 5s 15ms/step - loss: 6.2931 - accuracy: 0.0372
Epoch 7/50
313/313 [==============================] - 5s 15ms/step - loss: 5.9240 - accuracy: 0.0483
Epoch 8/50
313/313 [==============================] - 5s 14ms/step - loss: 5.5043 - accuracy: 0.0628
Epoch 9/50
313/313 [==============================] - 5s 15ms/step - loss: 5.0229 - accuracy: 0.0903
Epoch 10/50
313/313 [==============================] - 5s 15ms/step - loss: 4.4946 - accur

In [17]:
def autoCompletations(text, model):
    # Tokenization and Text vectorization
    text_sequences = np.array(tokenizer.texts_to_sequences([text])) # Pass the input text as a list
    # Pre-padding
    testing = pad_sequences(text_sequences, maxlen = Max_Sequence_Len-1, padding='pre')
    # Prediction
    y_pred_test = np.argmax(model.predict(testing,verbose=0))

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_pred_test:
            predicted_word = word
            break
    text += " " + predicted_word + '.'
    return text

In [33]:
def autoCompletations(text, model):
    # Tokenization and Text vectorization
    text_sequences = np.array(tokenizer.texts_to_sequences([text])) # Pass the input text as a list
    # Pre-padding
    testing = pad_sequences(text_sequences, maxlen = Max_Sequence_Len-1, padding='pre')
    # Prediction
    y_pred_test = np.argmax(model.predict(testing,verbose=0))

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
        if index == y_pred_test:
            predicted_word = word
            break
    text += " " + predicted_word + '.'
    return text

text_to_complete = "king is"
complete_sentence = autoCompletations( text_to_complete, model)
complete_sentence

'king is night.'